In [1]:
import json
import os
import mp
import numpy as np
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import soundfile as sf
import re

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [2]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [3]:
def chunk(alignment, reject = -8, minimum_length = 2.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls

In [4]:
data = defaultdict(list)
with open('prepared-pseudolabel.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        l = json.loads(l)
        data[l['audio_filename']].append((no, l))
        
len(data)

3085595it [00:11, 262609.70it/s]


1961155

In [5]:
rows = list(data.values())
len(rows)

1961155

In [6]:
!rm -rf prepared-pseudolabel-chunks
!mkdir prepared-pseudolabel-chunks

In [7]:
def loop(data):
    data, _ = data
    new_data = []
    for d in tqdm(data):
        
        aligns, scores = [], []
        for i in d:
            f = f'prepared-pseudolabel_alignment/{i[0]}.alignment'
            try:
                with open(f) as fopen:
                    align = json.load(fopen)
                    score = np.sum([s['score'] for s in align])
                    aligns.append(align)
                    scores.append(score)
            except:
                aligns.append([])
                scores.append(-9999)

        argmax = np.argmax(scores)
        no = d[argmax][0]
        text = d[argmax][1]['new_text']
        lang = text.split('<|startoftranscript|><|')[1].split('|')[0]
        cleaned_text = re.sub(r"<\|.*?\|>", "", text).strip()
        if lang_model.predict(cleaned_text)[0][0] == '__label__english':
            predict_lang = 'en'
        else:
            predict_lang = 'ms'
        
        chunks = chunk(aligns[argmax])
        audio_filename = d[argmax][1]['audio_filename']
        if len(chunks):
            y, sr = sf.read(audio_filename)
            for k, c in enumerate(chunks):
                
                skip = False
                for c_ in c:
                    if (c_['end'] - c_['start']) > 2:
                        skip = True
                        break
                if skip:
                    continue
                    
                new_f = os.path.join('prepared-pseudolabel-chunks', f'{no}-{k}.mp3')
                if not os.path.exists(new_f):
                    sf.write(new_f, y[int(sr * c[0]['start']): int(sr * c[-1]['end'])], sr)
                ts = []
                
                min_t = min([c_['start'] for c_ in c])
                
                for c_ in c:
                    start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                    w = c_['text']
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                ts = ''.join(ts)
                new_text = text = f"<|startoftranscript|><|{predict_lang}|><|transcribeprecise|>{ts}<|endoftext|>"
                new_data.append({
                    'audio_filename': new_f,
                    'new_text': new_text,
                })
                
    return new_data

In [8]:
r = loop((rows[:100], 0))
len(r)

100%|███████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.68it/s]


350

In [9]:
r[-1]

{'audio_filename': 'prepared-pseudolabel-chunks/181-4.mp3',
 'new_text': '<|startoftranscript|><|en|><|transcribeprecise|><|0.00|> untuk<|0.38|><|0.52|> subscribe.<|0.96|><|1.02|> Dan<|1.24|><|1.36|> kepada<|1.60|><|1.68|> anda,<|1.88|><|2.60|> anda<|2.74|><|2.84|> boleh<|3.14|><|3.32|> juga<|3.56|><|4.24|> mendengar<|4.86|><|endoftext|>'}

In [17]:
r = mp.multiprocessing(rows, loop, cores = 20)

100%|█████████████████████████████████████████████████████████████████████████████████| 98057/98057 [5:05:25<00:00,  5.35it/s]


In [18]:
len(r)

4775944

In [19]:
r[-10]

{'audio_filename': 'prepared-pseudolabel-chunks/3085589-0.mp3',
 'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> air.<|0.24|><|0.68|> Cara<|0.92|><|0.98|> tuang<|1.12|><|1.18|> air.<|1.28|><|1.36|> Presentation<|2.10|><|2.20|> kena<|2.34|><|2.38|> hafal.<|2.58|><|2.62|> Tak<|2.70|><|2.74|> boleh<|2.92|><|2.98|> terbalik.<|3.34|><|3.42|> Tumang<|3.74|><|3.78|> terbalik.<|4.12|><|4.32|> Ada<|4.46|><|4.54|> tak<|4.64|><|4.70|> you<|5.18|><|6.02|> curi-curi<|6.52|><|6.74|> tak<|6.84|><|6.90|> buat<|7.10|><|7.20|> apa<|7.32|><|7.40|> semua?<|7.60|><|8.24|> Tak<|8.34|><|8.40|> pernah<|8.64|><|8.68|> lagi<|8.86|><|8.96|> lah.<|9.10|><|9.24|> Sebab<|9.64|><|9.82|> kata<|10.04|><|endoftext|>'}

In [20]:
import IPython.display as ipd
ipd.Audio(r[-10]['audio_filename'])

In [21]:
with open('pseudolabel-whisper-word-timestamp.jsonl', 'w') as fopen:
    for r_ in r:
        fopen.write(f'{json.dumps(r_)}\n')
        fopen.flush()

In [22]:
!ls -lh pseudolabel-whisper-word-timestamp.jsonl

-rw-rw-r-- 1 husein husein 2.0G Jan   8 19:17 pseudolabel-whisper-word-timestamp.jsonl


In [23]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="pseudolabel-whisper-word-timestamp.jsonl",
    path_in_repo="pseudolabel-whisper-word-timestamp.jsonl",
    repo_id="mesolitica/Malaysian-STT-Whisper",
    repo_type="dataset",
)

pseudolabel-whisper-word-timestamp.jsonl:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/commit/71b9e77bed3d261ff4a842c94c12c85281f1b2f0', commit_message='Upload pseudolabel-whisper-word-timestamp.jsonl with huggingface_hub', commit_description='', oid='71b9e77bed3d261ff4a842c94c12c85281f1b2f0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-STT-Whisper'), pr_revision=None, pr_num=None)

In [24]:
!du -hs prepared-pseudolabel-chunks

157G	prepared-pseudolabel-chunks


In [ ]:
!zip -q -r -s 10000m prepared-pseudolabel-malaya-chunks.zip prepared-pseudolabel-malaya-chunks

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

for f in glob('prepared-pseudolabel-malaya-chunks.z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id="mesolitica/Malaysian-STT-Whisper",
        repo_type="dataset",
    )